In [43]:
!pip install xlsxwriter

In [44]:
import pandas as pd
import os
import json

In [45]:
# Read and convert place details to dataframe
with open('place-details-v3.json', 'r') as f:
    data_list = json.load(f)

names = []
price_levels = []
ratings = []
locations = [] # "lat,lng" -> expected format for solr 
deliveries = []
dine_ins = []
business_statuses = []
formatted_addresses = []
place_ids = []
user_ratings_totals = []
types = []

for data in data_list:
    names.append(data['name'])
    price_levels.append(data.get('price_level',None))
    ratings.append(data.get('rating',None))
    locations.append(f"{data['geometry']['location']['lat']},{data['geometry']['location']['lng']}")
    deliveries.append(data.get('delivery',False))
    dine_ins.append(data.get('dine_in',False))
    business_statuses.append(data.get('business_status',False))
    formatted_addresses.append(data.get('formatted_address', None))
    place_ids.append(data.get('place_id', None))
    user_ratings_totals.append(data.get('user_ratings_total', None))
    types.append(', '.join(data.get('types', [])))

places_df = pd.DataFrame({
    'RestaurantName': names,
    'PriceLevel': price_levels,
    'RestaurantRating': ratings,
    'LatLng': locations,
    'Delivery': deliveries,
    'Dine-in': dine_ins,
    'BusinessStatus': business_statuses,
    'FormattedAddress': formatted_addresses,
    'PlaceId': place_ids,
    'ReviewCount': user_ratings_totals,
    'Types': types
})

places_df.drop_duplicates(subset=['RestaurantName'], inplace=True)
places_df

,RestaurantName,PriceLevel,RestaurantRating,LatLng,Delivery,Dine-in,BusinessStatus,FormattedAddress,PlaceId,ReviewCount,Types
0,Alati Divine Greek Cuisine,3.0,4.5,"1.2803412,103.8467476",False,True,OPERATIONAL,"73 Amoy St, Singapore 069892",ChIJn0mYZQ0Z2jERomxAoOxa4zI,695.0,"restaurant, food, point_of_interest, establish..."
1,Alma by Juan Amador,4.0,4.5,"1.308128,103.8341296",True,True,OPERATIONAL,"22 Scotts Rd, Singapore 228221",ChIJtSfORu0Z2jERaTv_fbShVmM,487.0,"restaurant, food, point_of_interest, establish..."
2,Altro Zafferano,3.0,4.4,"1.2832397,103.8520217",False,True,OPERATIONAL,"10 Collyer Quay, Level 43, Singapore 049315",ChIJ8Ts47w4Z2jER3wgHYemC4hw,951.0,"bar, restaurant, food, point_of_interest, esta..."
3,Art di Daniele Sperindio,3.0,4.5,"1.2907066,103.8519791",False,True,OPERATIONAL,"1 St Andrew’s Road, 06-02 National Gallery, Si...",ChIJmzR_VQkZ2jERYb8SzlA6m0w,589.0,"bar, restaurant, point_of_interest, food, esta..."
4,Artemis Grill & Sky Bar,4.0,4.5,"1.2818026,103.8501248",False,True,OPERATIONAL,"138 Market St, #40-01 CapitaGreen, Singapore 0...",ChIJq_pn0w0Z2jERJ9ThzorUL6A,1283.0,"bar, restaurant, point_of_interest, food, esta..."
...,...,...,...,...,...,...,...,...,...,...,...
117,Yan Restaurant at National Gallery Singapore 宴中餐馆,3.0,4.1,"1.2902783,103.8517729",True,True,OPERATIONAL,"1 St Andrew's Rd, #05-02 National Gallery, Sin...",ChIJLxx8T6cZ2jERXDPzASEmG6k,594.0,"restaurant, point_of_interest, food, establish..."
118,Yardbird Southern Table and Bar,3.0,4.2,"1.2830591,103.8580401",True,True,OPERATIONAL,"2 Bayfront Ave, # B1 - 07, Singapore 018972",ChIJn9YqwQUZ2jERQd1xVpCATwk,2002.0,"restaurant, bar, point_of_interest, food, esta..."
119,Yellow Pot Tanjong Pagar,NaN,4.1,"1.2786983,103.84319",True,True,OPERATIONAL,"83 Duxton Rd, Duxton Reserve, Singapore 089540",ChIJP_zH9VkZ2jERNfJDM0o0EwE,247.0,"restaurant, food, point_of_interest, establish..."
120,Zén,4.0,4.6,"1.2794687,103.8403607",True,True,OPERATIONAL,"41 Bukit Pasoh Rd, Singapore 089855",ChIJpxlhJpkZ2jER7gzIjfgz8co,221.0,"restaurant, food, point_of_interest, establish..."


In [46]:
folder_path = "raw/reviews/"

dfs = []

for index, file in enumerate(sorted(os.listdir(folder_path))):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        restaurant_name = file.replace("reviews_for_", "").replace(".csv", "")
        df = pd.read_csv(file_path)
        df['Restaurant'] = restaurant_name
        dfs.append(df)

result_df = pd.concat(dfs, ignore_index=True)
result_df = result_df.iloc[:,1:]
result_df

,Name,Rating,Review,Restaurant
0,A S,5 stars,Loved the ambience !,Alati Divine Greek Cuisine
1,Aaron Tan,5 stars,"Food was great . Salted baked fish , prawns an...",Alati Divine Greek Cuisine
2,Aaron Y,5 stars,The food is spectacular. We were sharing about...,Alati Divine Greek Cuisine
3,Abber Tan,5 stars,Yummy authentic Greek food. The dips were real...,Alati Divine Greek Cuisine
4,Adam Siew,4 stars,Pretty good food. =),Alati Divine Greek Cuisine
...,...,...,...,...
59186,Luca Cagnolati,4 stars,The chef has changed but the cuisine remains i...,il Cielo
59187,Yan N,2 stars,Perhaps because the Japanese chef was not pres...,il Cielo
59188,sweet freedom,5 stars,That's fine,il Cielo
59189,Kuswati Warnisa,5 stars,You must be very happy,il Cielo


In [47]:
combined_df = pd.merge(result_df, places_df, how='left', left_on='Restaurant', right_on='RestaurantName')
combined_df['Rating'] = combined_df['Rating'].str.extract(r'(\d+) star').astype(float)
combined_df.drop(columns=['Restaurant'], inplace=True)
combined_df.to_csv("raw-v3/combined.csv", index=False)

In [48]:
num_rows = len(combined_df)
num_words = combined_df['Review'].str.split().apply(len).sum()
print("Number of records:", num_rows)
print("Number of words:", num_words)

Number of records: 59191
Number of words: 1454999


In [49]:
combined_df['Label'] = pd.cut(combined_df['Rating'], bins=[-float('inf'), 2.9, 3.1, float('inf')], labels=['NEGATIVE', 'NEUTRAL', 'POSITIVE'])
combined_df.to_csv("raw-v3/combined_with_labels.csv", index=False)
combined_df.to_json("final-v3.json", orient='records')

In [50]:
label_counts = combined_df['Label'].value_counts()
label_counts

POSITIVE    49758
NEGATIVE     5162
NEUTRAL      4271
Name: Label, dtype: int64

In [51]:
positive_df = combined_df[combined_df['Label'] == 'POSITIVE'].sample(n=500, random_state=42)
negative_df = combined_df[combined_df['Label'] == 'NEGATIVE'].sample(n=500, random_state=42)
eval_df = pd.concat([positive_df, negative_df], ignore_index=True)
eval_df.to_excel("raw-v3/eval.xls", index=False, engine='xlsxwriter')

In [52]:
label_counts = eval_df['Label'].value_counts()
label_counts

NEGATIVE    500
POSITIVE    500
NEUTRAL       0
Name: Label, dtype: int64